# Equipment

| Equipment | Description | Serial Number | Date of last calibration |
|-----------|-------------|---------------|--------------------------|
| Multimeter | Mastech M92 | 20011206033 | - |
| Multimeter | Fluke 8010A | 2047249 | - |
| Oscilloscope | Siglent SDS2354X HD |  SDS2HBAQ6R0257 | TBD |
| Power Supply | Siglent SPD3303C | SPD3EEEC6R0513 | TBD |
| Electronic Load | Siglent SDL1020X-E | SDL13GCC6R0182 | TBD |

# Test Incident Reports

| Test | Incident | Proposed Solution | Status |
|------|----------|-------------------|--------|
| [TIR1](#tir1)<a id="ttir1"/> | improve detector tuning range | • replace C5 by 20pF trimmer<br/>• remove C7 |  |
| [TIR2](#tir2)<a id="ttir2"/> | audio jacks too loose | decrease size of mounting holes to ⌀0.9mm or ⌀1.0mm |  |
| [TIR3](#tir3)<a id="ttir3"/> | high noise on audio input J7 | add 100nF capacitor from J7.2 to ground and J7.3 to ground on PCB |  |
| [TIR4](##tir4)<a id="ttir4"/> | audio output too loud at maximum volume | [see details below](#potmeter-fix) |  |

# Test Results
## Detector Circuit
| Action | Expected Result | Observed result | Status |
|--------|-----------------|-----------------|--------|
| • use spectrum analyser to create an FM modulated signal<br/>• fc=10.7MHz, fm=1kHz, m=15kHz, P=-50dBm | 1kHz output wave on node R1/C11 on oscilloscope | • 1k output visible<br/>• doesn't look very clean, but that may be due to the SA<br/><img src="./doc/screenshots/SDS2354X_HD_PNG_1.png" />| ✅ |
| tune C5 for maximum demodulated audio amplitude <br/> | ? | 106mVpp | ✅ |
| replace C5+C6+C7 with single capacitor of same value  | ? | [see details below](#replace-c5)<br/>add test point for detector tuning on R1/C11 | 🛑<br/>[TIR1](#ttir1)<a id="tir1"/> |
| • repeat the previous test using the Siglent signal generator instead of the SA<br/>• output set to 5mVrms (-33dBm) | less harmonics on demodulated audio | <img src="./doc/screenshots/SDS2354X_HD_PNG_2.png" /> | ✅ |
| Measure DC-voltage between R7 and R11 | 3.79Vdc |3.74Vdc | ✅ |
| Measure DC-bias on base of Q1 | 3.77Vdc | 3.68Vdc | ✅ |
| Measure DC-voltage on base of Q2 | 3.18Vdc | 3.12Vdc| ✅ |
| Measure DC-voltage on emitter of Q2 | 2.49Vdc | 2.41Vdc| ✅ |
| Mounting of audio jacks | fit well | too loose fit | 🛑<br/>[TIR2](#ttir2)<a id="tir2"/> |
| Output level at 75kHz frequency deviation | 160mVrms | 160mVrms | ✅ |

## Buffer amplifier
### Test Setup

While R8 is not yet populated, we can use an easy test setup to verify the buffer amplifier performance.  We use REW (Room EQ Wizard) to generate a sine wave at 1kHz, which is fed into the audio input J7.  The output is measured at J5.3 using the oscilloscope.  A 100nF capacitor is soldered from audio input to ground near J7 to provide HF noise filtering.

<figure>
<img src="./doc/IMG_20260103_141616_264.jpg" width="800"/>
<figcaption>Test setup for buffer amplifier</figcaption>
</figure>
<figure>
<img src="./doc/IMG_20260103_141625_667.jpg" width="800"/>
<figcaption>Close-up of buffer amplifier test setup, with 100nF capacitor soldered from audio input to ground near J7.</figcaption>
</figure>

| Action | Expected Result | Observed result | Status |
|--------|-----------------|-----------------|--------|
| Measure distortion at maximum input level (160mVrms)<br/>Use REW's generator to generate 1kHz sine wave<br/>Use REW's RTA window to measure | <1% | 0.1% | ✅ |
| Measure output level at maximum input level (160mVrms) | ~160mVrms | 157mVrms | ✅ |
| Noise on J7 | low, doesn't hamper oscilloscope measurements | too high to measure small signals on oscilloscope | 🛑<br/>[TIR3](#ttir3)<a id="tir3"/> |
| Frequency response | flat from 20Hz to 20kHz within 3dB | <img src="./doc/buffer_amplifier_f_response.png" /><br/>The trace is noisy because of the noisy calibration file | ✅ |
| Output level on R1/C11 when volume is set to minimum | ~160mVrms | 160mVrms | ✅ |
| Output level on R1/C11 when volume is set to maximum | ~116mVrms | 123mVrms | ✅ |
| Output level on J5.3 when volume is set to minimum and 32ohm headset is connected | Zout=30Ω → 80mVrms | 86mVrms | ✅ |
| Output volume in headset when volume is set to maximum and 32ohm headset is connected | comfortable listening level | too loud<br/>output volume should be limited to 5mVrms | 🛑<br/>[TIR4](##ttir4)<a id="tir4"/>|

# Test Result Details
## Find suitable capacitor value to replace C5+C6+C7 <a id="replace-c5"></a>
* C5=0pF, C6=82pF, C7=10pF → Vpp=85mV
* C5=0pF, C6=82pF, C7=12pF → Vpp=102mV
* C5=0pF, C6=82pF, C7=15pF → Vpp=102mV
* C5=0pF, C6=82pF, C7=16pF → Vpp=96mV
* C5=0pF, C6=82pF, C7=22pF → Vpp=29mV

It comes down to a few picofarads, so we better keep a trimmer in place.  We could use a 20pF trimmer for C5 and leave C7 out.  As such, we allow for some tolerance on the value of L1.

## Find solution for loud audio output at maximum volume <a id="potmeter-fix"></a>
Only a small portion of the usable range of the volume potentiometer can be used because at maximum volume the output is way too loud for a headset.  If you accidentally (or someone else on purpose) turn the volume knob to maximum, you may damage your hearing.

### Considerations
1. It's undesirable to have a high impedance in the "AUDIO" net feeding into the next stage.  
2. It's undesirable to add impedance in the ground lead because of noise pick-up.

### Proposed solution
So let's try this:
1. We could replace R18 and R19 by a single resistor of 100Ω.  That is still much lower than the 2.4kΩ input impedance of the next stage.
2. Tie pin 2 & 3 of J5 together to have audio on both ears.  
3. Add a 18Ω resistor from J5.5 to J5.2 and J5.3 to limit the output voltage.  Pin 5 of J5 will only be connected to ground (J5.1) when a jack is inserted.  Without the jack, it's out of the circuit and doesn't load down the output.

The maximum volume with a 36Ω headset will then be limited to about 10mVrms, which is still very loud, but acceptable for short listening tests.